Tokenizar - Unicode y evaluación de librerías

In [1]:
# Instalamos las librerías necesarias (si no están instaladas)
!pip install PyMuPDF spacy nltk stanza transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590

In [4]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 44.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import fitz  # PyMuPDF para leer PDF
import unicodedata
import spacy
import nltk
import stanza
from transformers import AutoTokenizer

In [5]:
# Descargar modelos necesarios
nltk.download('punkt')
stanza.download('es')
nlp_stanza = stanza.Pipeline('es')
nlp_spacy = spacy.load('es_core_news_sm')
tokenizer_bert = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: es (Spanish) ...
INFO:stanza:File exists: /root/stanza_resources/es/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor    | Package           |
------------------------------------
| tokenize     | combined          |
| mwt          | combined          |
| pos          | combined_charlm   |
| lemma        | combined_nocharlm |
| constituency | combined_charlm   |
| depparse     | combined_charlm   |
| sentiment    | tass2020_charlm   |
| ner          | conll02           |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a toke

tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [7]:
# 1. Leer el archivo PDF
def leer_pdf(ruta):
    with fitz.open(ruta) as doc:
        texto = "\n".join([page.get_text("text") for page in doc])
    return texto

ruta_pdf = "/content/texto 1.1.pdf"
texto = leer_pdf(ruta_pdf)
print("Texto original:\n", texto[:500])

Texto original:
 El lenguaje es una de las herramientas centrales en nuestra vida social y profesional. 
Entre otras cosas, actúa como un medio para transmitir ideas, información, opiniones 
y sentimientos; así como para persuadir, pedir información, o dar ordenes. Asimismo, 
el lenguaje humano es algo que esta en constante cambio y evolución; y que puede 
llegar a ser muy ambiguo y variable. Tomemos por ejemplo la frase "comí una pizza con 
amigos" comparada con "comí una pizza con aceitunas"; su estructura es 


In [8]:
# 2. Convertir a minúsculas
texto_minusculas = texto.lower()
print("\nTexto en minúsculas:\n", texto_minusculas[:500])


Texto en minúsculas:
 el lenguaje es una de las herramientas centrales en nuestra vida social y profesional. 
entre otras cosas, actúa como un medio para transmitir ideas, información, opiniones 
y sentimientos; así como para persuadir, pedir información, o dar ordenes. asimismo, 
el lenguaje humano es algo que esta en constante cambio y evolución; y que puede 
llegar a ser muy ambiguo y variable. tomemos por ejemplo la frase "comí una pizza con 
amigos" comparada con "comí una pizza con aceitunas"; su estructura es 


In [11]:
# 3. Normalización Unicode (NFC)
def normalizar_unicode(texto):
    return unicodedata.normalize('NFC', texto)

texto_normalizado = normalizar_unicode(texto_minusculas)
print("\nTexto normalizado:\n", texto_normalizado[:500])


Texto normalizado:
 el lenguaje es una de las herramientas centrales en nuestra vida social y profesional. 
entre otras cosas, actúa como un medio para transmitir ideas, información, opiniones 
y sentimientos; así como para persuadir, pedir información, o dar ordenes. asimismo, 
el lenguaje humano es algo que esta en constante cambio y evolución; y que puede 
llegar a ser muy ambiguo y variable. tomemos por ejemplo la frase "comí una pizza con 
amigos" comparada con "comí una pizza con aceitunas"; su estructura es 


In [19]:
# Download the necessary data for Spanish tokenization
nltk.download('punkt')
nltk.download('spanish_grammars')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package spanish_grammars to /root/nltk_data...
[nltk_data]   Unzipping grammars/spanish_grammars.zip.


True

In [20]:
# 4. Tokenización con diferentes librerías

def tokenizar_spacy(texto):
    return [token.text for token in nlp_spacy(texto)]

def tokenizar_nltk(texto):
    return nltk.word_tokenize(texto, language='spanish')

def tokenizar_stanza(texto):
    doc = nlp_stanza(texto)
    return [word.text for sent in doc.sentences for word in sent.words]

def tokenizar_bert(texto):
    return tokenizer_bert.tokenize(texto)

In [21]:
# Aplicamos cada método
tokens_spacy = tokenizar_spacy(texto_normalizado)
tokens_nltk = tokenizar_nltk(texto_normalizado)
tokens_stanza = tokenizar_stanza(texto_normalizado)
tokens_bert = tokenizar_bert(texto_normalizado)

Token indices sequence length is longer than the specified maximum sequence length for this model (1945 > 512). Running this sequence through the model will result in indexing errors


In [22]:
# Mostramos los primeros 20 tokens de cada método
print("\nTokens con SpaCy:", tokens_spacy[:20])
print("\nTokens con NLTK:", tokens_nltk[:20])
print("\nTokens con Stanza:", tokens_stanza[:20])
print("\nTokens con BERT:", tokens_bert[:20])


Tokens con SpaCy: ['el', 'lenguaje', 'es', 'una', 'de', 'las', 'herramientas', 'centrales', 'en', 'nuestra', 'vida', 'social', 'y', 'profesional', '.', '\n', 'entre', 'otras', 'cosas', ',']

Tokens con NLTK: ['el', 'lenguaje', 'es', 'una', 'de', 'las', 'herramientas', 'centrales', 'en', 'nuestra', 'vida', 'social', 'y', 'profesional', '.', 'entre', 'otras', 'cosas', ',', 'actúa']

Tokens con Stanza: ['el', 'lenguaje', 'es', 'una', 'de', 'las', 'herramientas', 'centrales', 'en', 'nuestra', 'vida', 'social', 'y', 'profesional', '.', 'entre', 'otras', 'cosas', ',', 'actúa']

Tokens con BERT: ['el', 'lenguaje', 'es', 'una', 'de', 'las', 'herramientas', 'centrales', 'en', 'nuestra', 'vida', 'social', 'y', 'profesional', '.', 'entre', 'otras', 'cosas', ',', 'actúa']


**Mejorar Bert**

In [23]:
tokens_bert = tokenizer_bert.tokenize(texto_normalizado)

# Dividimos en fragmentos de 512 tokens
max_length = 512
chunks = [tokens_bert[i:i+max_length] for i in range(0, len(tokens_bert), max_length)]

print(f"Se generaron {len(chunks)} fragmentos de 512 tokens")

Se generaron 4 fragmentos de 512 tokens


In [25]:
def tokenizar_bert(texto):
    return tokenizer_bert.tokenize(texto)

In [26]:
print("\nTokens con BERT:", tokens_bert[:20])


Tokens con BERT: ['el', 'lenguaje', 'es', 'una', 'de', 'las', 'herramientas', 'centrales', 'en', 'nuestra', 'vida', 'social', 'y', 'profesional', '.', 'entre', 'otras', 'cosas', ',', 'actúa']
